In [1]:
from __future__ import print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from PIL import Image
from sklearn import metrics
%matplotlib

Using matplotlib backend: Qt5Agg


In [2]:
data_transforms = {
    'train': transforms.Compose([transforms.Resize((224,224)), transforms.ToTensor(),
                                 transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]),
    'test': transforms.Compose([transforms.Resize((224,224)), transforms.ToTensor(),
                               transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]),}

data_dir = r'D:/Documents/Free_Lence_Projects/Anas_Project/org_DataSet_res/'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'test']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=16,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test']}
class_names = image_datasets['train'].classes
print(class_names)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

['Benign', 'Cancer', 'Normal']


In [3]:
Training_loss = []
Test_loss = []
Training_accurecy = []
Test_accurecy = []



def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and testing phase
        for phase in ['train', 'test']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0
            
            
            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
#                         scheduler.step()

                        
        
                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
            
            if phase == 'train':
                Training_loss.append(epoch_loss)
                Training_accurecy.append(epoch_acc)
            # deep copy the model
            if phase == 'test':
                Test_loss.append(epoch_loss)
                Test_accurecy.append(epoch_acc)
                if epoch_acc > best_acc:
                    best_acc = epoch_acc
                    best_model_wts = copy.deepcopy(model.state_dict())
        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best Testing Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model,Training_loss

In [4]:
# To Create your own architactue u need to inhrit nn.Module . This gives your class all proprties of any Neural netwokr
class VGG(nn.Module):
    
    # initilization function to create the model 
    def __init__(self):
        super(VGG,self).__init__()

        
        # vgg net
        self.vgg_net = models.vgg16_bn(pretrained=True)
        self.vgg_net.classifier = nn.Sequential(nn.Linear(in_features=25088, out_features=4096, bias=True)
        ,nn.ReLU(inplace=True)
        ,nn.Dropout(p=0.5, inplace=False)
        ,nn.Linear(in_features=4096, out_features=4096, bias=True)
        ,nn.ReLU(inplace=True)
        ,nn.Dropout(p=0.5, inplace=False),nn.Linear(in_features=4096, out_features=3, bias=True))

        
    # this function is called automatically for forward propagation    
    def forward(self, x):
        output = self.vgg_net(x)
        return output
    
    
# To Create your own architactue u need to inhrit nn.Module . This gives your class all proprties of any Neural netwokr
class AlexNet(nn.Module):
    
    # initilization function to create the model 
    def __init__(self):
        super(AlexNet,self).__init__()
        
        # Alex Net 
        self.AlexNet = models.alexnet(pretrained=True)
        self.AlexNet.classifier = nn.Sequential(nn.Linear(in_features=9216, out_features=4096, bias=True)
        ,nn.ReLU(inplace=True)
        ,nn.Dropout(p=0.5, inplace=False)
        ,nn.Linear(in_features=4096, out_features=4096, bias=True)
        ,nn.ReLU(inplace=True)
        ,nn.Dropout(p=0.5, inplace=False),nn.Linear(in_features=4096, out_features=3, bias=True))

        
    # this function is called automatically for forward propagation    
    def forward(self, x):
        output = self.AlexNet(x)
        return output
    
    
# To Create your own architactue u need to inhrit nn.Module . This gives your class all proprties of any Neural netwokr
class ResNet(nn.Module):
    
    # initilization function to create the model 
    def __init__(self):
        super(ResNet,self).__init__()
        
        # Alex Net 
        self.ResNet = models.resnet50(pretrained=True)
        self.ResNet.fc = nn.Sequential(nn.Linear(in_features=2048, out_features=4096, bias=True)
        ,nn.ReLU(inplace=True)
        ,nn.Dropout(p=0.5, inplace=False)
        ,nn.Linear(in_features=4096, out_features=4096, bias=True)
        ,nn.ReLU(inplace=True)
        ,nn.Dropout(p=0.5, inplace=False),nn.Linear(in_features=4096, out_features=3, bias=True))

        
    # this function is called automatically for forward propagation    
    def forward(self, x):
        output = self.ResNet(x)
        return output

In [5]:
criterion = nn.CrossEntropyLoss()
# Observe that all parameters are being optimized
smodels = ['AlaxNet','VGG','ResNet']
dic_graph = {}
res_arries = ['testACC','trainACC','testLoss','trainLoss']

In [6]:
for model in smodels:
    torch.cuda.empty_cache()                            
    if model == 'AlaxNet':
        model_ft = AlexNet()  
    elif model == 'VGG':
        model_ft = VGG()
    elif model == 'ResNet':
        model_ft = ResNet()
    model_ft = model_ft.to(device)
    optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.00001)
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)
    model_ft,Training_loss = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=10)
    torch.save(model_ft.state_dict(), f'{model}.pth.tar')
    dic_graph[model+res_arries[3]] = Training_loss.copy()
    dic_graph[model+res_arries[2]] = Test_loss.copy()
    dic_graph[model+res_arries[1]] = Training_accurecy.copy()
    dic_graph[model+res_arries[0]] = Test_accurecy.copy()
    print(f"The {model} is finished Training")
    Training_loss = []
    Test_loss = []
    Training_accurecy = []
    Test_accurecy = []

Epoch 0/9
----------


C:\Users\Abdelsalam\anaconda3\lib\site-packages\torch\optim\lr_scheduler.py:118: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


train Loss: 0.8154 Acc: 0.5850
test Loss: 0.7174 Acc: 0.6081

Epoch 1/9
----------
train Loss: 0.6748 Acc: 0.6675
test Loss: 0.7079 Acc: 0.6206

Epoch 2/9
----------
train Loss: 0.5999 Acc: 0.7204
test Loss: 0.7213 Acc: 0.6105

Epoch 3/9
----------
train Loss: 0.5379 Acc: 0.7570
test Loss: 0.7872 Acc: 0.6027

Epoch 4/9
----------
train Loss: 0.4629 Acc: 0.8031
test Loss: 0.7662 Acc: 0.6386

Epoch 5/9
----------
train Loss: 0.3869 Acc: 0.8422
test Loss: 0.7126 Acc: 0.6682

Epoch 6/9
----------
train Loss: 0.2840 Acc: 0.8969
test Loss: 0.7481 Acc: 0.6628

Epoch 7/9
----------
train Loss: 0.2579 Acc: 0.9124
test Loss: 0.7668 Acc: 0.6604

Epoch 8/9
----------
train Loss: 0.2454 Acc: 0.9144
test Loss: 0.7628 Acc: 0.6659

Epoch 9/9
----------
train Loss: 0.2328 Acc: 0.9240
test Loss: 0.7758 Acc: 0.6596

Training complete in 3m 43s
Best Testing Acc: 0.668228
The AlaxNet is finished Training
Epoch 0/9
----------
train Loss: 0.7593 Acc: 0.6194
test Loss: 0.6476 Acc: 0.6690

Epoch 1/9
----------

## Traning and Testing Graph

In [7]:
# summarize history for accuracy
for m in smodels:
    plt.plot(dic_graph[m+res_arries[1]],label=m+"_"+res_arries[1])
    plt.plot( dic_graph[m+res_arries[0]],label=m+"_"+res_arries[0])

plt.title('models accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(loc='upper left')
plt.show()

In [ ]:
dic_graph

In [9]:
# summarize history for accuracy
for m in smodels:
    plt.plot(dic_graph[m+res_arries[2]],label=m+"_"+res_arries[2])
    plt.plot( dic_graph[m+res_arries[3]],label=m+"_"+res_arries[3])
plt.title('models Loss')
plt.ylabel('Loss')
plt.xlabel('epoch')
plt.legend(loc='upper left')
plt.show()

## confusion_matrix for all models

In [10]:
from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd
model_ft = None
roc_graphs = {}
y_pred = []
y_true = []
# iterate over test data
for stage in ['test','train']:
    y_pred = []
    y_true = []
    for m in smodels:
        torch.cuda.empty_cache()                            
        if m == 'AlaxNet':
            model_ft = AlexNet()  
        elif m == 'VGG':
            model_ft = VGG()
        elif m == 'ResNet':
            model_ft = ResNet()
        
        print(m)
        print(model_ft)
        model_ft = model_ft.to(device)
        model_ft.load_state_dict(torch.load(f'{m}.pth.tar'))
        
        for inputs, labels in dataloaders[stage]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                output = model_ft(inputs) # Feed Network

                output = (torch.max(torch.exp(output), 1)[1]).data.cpu().numpy()
                y_pred.extend(output) # Save Prediction

                labels = labels.data.cpu().numpy()
                y_true.extend(labels) # Save Truth

        fpr, tpr, _ = metrics.roc_curve(y_true,  y_pred,pos_label=1)
#         auc = metrics.roc_auc_score(y_true,  y_pred,pos_label=1)

        roc_graphs[m+stage+"fpr"] = fpr.copy()
        roc_graphs[m+stage+"tpr"] = tpr.copy()
#         roc_graphs[m+stage+"auc"] = auc


        classes = class_names

        # Build confusion matrix
        cf_matrix = confusion_matrix(y_true, y_pred,)
        df_cm = pd.DataFrame(cf_matrix/np.sum(cf_matrix) *3, index = [i for i in classes],
                             columns = [i for i in classes])
        plt.figure(figsize = (12,7))
        plt.xlabel("Predection")
        sn.heatmap(df_cm, annot=True)
        plt.savefig(f'ConfusionMatrex_{m}_{stage}.png')

AlaxNet
AlexNet(
  (AlexNet): AlexNet(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
      (1): ReLU(inplace=True)
      (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (4): ReLU(inplace=True)
      (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (7): ReLU(inplace=True)
      (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (9): ReLU(inplace=True)
      (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
    (classifier): Sequential(
      (0): Linear(in_features=9216, out_

AlaxNet
AlexNet(
  (AlexNet): AlexNet(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
      (1): ReLU(inplace=True)
      (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (4): ReLU(inplace=True)
      (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (7): ReLU(inplace=True)
      (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (9): ReLU(inplace=True)
      (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
    (classifier): Sequential(
      (0): Linear(in_features=9216, out_

## Draw ROC 

In [11]:
stage = 'test'
for m in smodels:
    plt.plot(roc_graphs[m+stage+"fpr"],roc_graphs[m+stage+"tpr"], label=f'{m}_{stage}')
    

plt.title('ROC for Testing Dataset')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend()
plt.show()

In [12]:
stage = 'train'
for m in smodels:
    plt.plot(roc_graphs[m+stage+"fpr"],roc_graphs[m+stage+"tpr"], label=f'{m}_{stage}')

plt.title('ROC for Training Dataset')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend()
plt.show()